# Identity Verification Getting Started

-------

This lab simply checks to make sure that your environment is setup to run the identity verification labs. 


## Introduction 

-------

In this notebook, we'll check our environment and client connectivity prior to running the Identity Verification labs.  


### Check the Python Environment is Setup
-----
Make sure that the following block of code will run, make any changes to your region, bucket and clients necessary

In [ ]:

import boto3
from IPython.display import HTML, display, Image as IImage


mySession = boto3.session.Session()
aws_region = mySession.region_name
bucket_name =  "identity-verification-media-123"
print("AWS Bucket: {}".format(bucket_name))

### Setup Clients 
-----
Here we are going to use both S3 and Rekognition clients  

In [ ]:
s3_client  = boto3.client('s3')
rek_client = boto3.client('rekognition')

### Display a Face from S3
-------
Here we are going to display the face of Al Gore

In [ ]:

## Image of a Face
face_image = "Al_Gore_0001.png"
print(face_image)
display(IImage(url=s3_client.generate_presigned_url('get_object', 
                                                    Params={'Bucket': bucket_name, 
                                                            'Key'   : face_image})))

###  This function will list the first 1k files stored in our bucket. 
-----
We'll make use of these files in subsequent labs. 

In [ ]:
# there should be close to 1000 files in the bucket. 
def list_s3_files_using_client(bucket_name):
    """
    This functions list all files in s3 bucket.
    :return: None
    """
    print(f"Files located in bucket: {bucket_name}")
    s3_client = boto3.client("s3")
    response = s3_client.list_objects_v2(Bucket=bucket_name)
    files = response.get("Contents")
    for file in files:
        print(f"file_name: {file['Key']}, size: {file['Size']}")

# -- run our function 
list_s3_files_using_client(bucket_name)

## Check Rekognition APIs
-----
Here we'll use the detect_faces API to ensure that our Rekognition client is properly configured. the following should produce a FaceDetails object. 


In [ ]:
response = rek_client.detect_faces(Image={'S3Object':{
    'Bucket': bucket_name,
    'Name'  : face_image}},
    Attributes=['ALL'])

response